## Importing librarie

In [1]:
#Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path 

#Importing libraries to connect to the database
import auth
import getpass

user  = getpass.getuser()
token = auth.get_valid_token()

#Connecting to Datalake
import trino
import os
connection = trino.dbapi.connect(
    host=os.environ['PRESTO_HOST'],
    port=443,
    user=user,
    catalog="hive",
    schema="dwh_insider",
    http_scheme='https',
    auth=trino.auth.BasicAuthentication(user, token))

In [2]:
##Loading the data from the Feige team 
La_poste_transportation_costs = pd.read_excel('/home/vralaptisrin/nfs/notebooks/Varshitha_notebooks/France analysis/LaPosté_Overview_2022.xlsx')
La_poste_transportation_costs

,Invoice,Month,Sum
0,6700248191,1.0,6587932.21
1,6700249483,2.0,4127629.48
2,6700250810,3.0,2766610.52
3,6700252129,4.0,3117427.09
4,6700253445,5.0,3902195.37
5,6700254764,6.0,4697008.58
6,7570440295,7.0,5833231.07
7,7570442329,8.0,2919726.63
8,7570444665,9.0,2710169.31
9,7570447016,10.0,3690299.39


In [ ]:
# Taking the claim information from the datalake
query = """
with ffsoi_base as (
        select

                        ffsoi.shipment_id, ffsoi.ts_est_delivery_from, ffsoi.ts_est_delivery_to,
                        ffsoi.sk_order_type, ffsoi.sk_logistic_provider, ffsoi.sk_order_date,
                        ffsoi.full_cancellation_flag, ffsoi.sk_selected_fulfill_service,
                        ffsoi.sk_delivery_type, ffsoi.sk_sales_channel, ffsoi.sk_destination,
                        ffsoi.sk_wmo_flag, ffsoi.shipment_has_ldd, ffsoi.sk_stock_location_id,
                        ffsoi.sk_carrier,

                        fccs.ts_first_delivery_attempt, fccs.sk_customer_delivery_date,
                        fccs.ts_customer_delivery,
                        fccs.lt_min_order_to_fda_wo_sun, fccs.lt_min_order_to_fda_wo_wend,
                        fccs.sk_delivery_cluster_wo_sun, fccs.is_prepayment_shipment,

                        fccs.is_delivered_too_early,
                        fccs.is_delivered_two_days_earlier,
                        fccs.is_delivered_a_day_earlier,
                        fccs.is_delivered_as_promised,
                        fccs.is_home_target,
                        fccs.is_pickup_point_target,
                        fccs.is_delivered_to_home,
                        fccs.is_delivered_to_pickup_point,

                        coalesce(ffsoi.sk_carrier, fccs.sk_carrier) as sk_carrier,
                        coalesce(ffsoi.sk_delivery_product, fccs.sk_delivery_product) as sk_delivery_product,

                        f_serv.fulfillment_service,
                        d_serv.service_name,
                        d_prod_serv.service_name as del_serv,
                        chan.sk_cfa,
                        country.destination_country,
                        logistic.logistic_provider_name,
                        logistic.logistic_provider,
                        stock.warehouse_name,
                        carr.carrier_description,
                        d_prod.sk_delivery_service,
                        d_prod.sk_delivery_carrier,
                        case when chan.sk_cfa in (31, 26) then 'FS'
                             when chan.sk_cfa = 28 then 'Lounge' end as cfa -- 26 is empty for recent periods, it stands for zlash


        from

                      (select shipment_id,
                              min(ts_est_delivery_from) as ts_est_delivery_from,
                              min(ts_est_delivery_to) as ts_est_delivery_to,
                              max(sk_order_type) as sk_order_type,
                              max(sk_logistic_provider) as sk_logistic_provider,
                              max(sk_order_date) as sk_order_date,
                              max(full_cancellation_flag) as full_cancellation_flag,
                              max(sk_selected_fulfill_service) as sk_selected_fulfill_service,
                              max(sk_delivery_type) as sk_delivery_type,
                              max(sk_sales_channel) as sk_sales_channel,
                              max(sk_destination) as sk_destination,
                              max(sk_wmo_flag) as sk_wmo_flag,
                              max(sk_stock_location_id) as sk_stock_location_id,
                              max(is_long_distance_item) as shipment_has_ldd,
                              max(sk_carrier) as sk_carrier,
                              max(sk_delivery_product) as sk_delivery_product
                      from dwh_confidential.f_fos_shipment_order_items
                      where sk_order_date between  20220101 and 20221231
                      group by shipment_id ) as ffsoi

                      -- For calculated lead times [sk_delivery_cluster_wo_sun, lt_min_order_to_fda_wo_sun,
                      -- lt_min_order_to_fda_wo_wend]
                      left join (select shipment_id, 
                                        max(ts_first_delivery_attempt) as ts_first_delivery_attempt,
                                        max(sk_first_delivery_attempt_date) as sk_first_delivery_attempt_date,
                                        max(ts_customer_delivery) as ts_customer_delivery,
                                        max(sk_customer_delivery_date) as sk_customer_delivery_date,
                                        max(lt_min_order_to_fda_wo_sun) as lt_min_order_to_fda_wo_sun,
                                        max(lt_min_order_to_fda_wo_wend) as lt_min_order_to_fda_wo_wend,
                                        max(sk_delivery_cluster_wo_sun) as sk_delivery_cluster_wo_sun,
                                        max(sk_carrier) as sk_carrier,
                                        max(sk_delivery_product) as sk_delivery_product,
                                        max(is_delivered_too_early) as is_delivered_too_early,
                                        max(is_delivered_two_days_earlier) as is_delivered_two_days_earlier,
                                        max(is_delivered_a_day_earlier) as is_delivered_a_day_earlier,
                                        max(is_delivered_as_promised) as is_delivered_as_promised,
                                        max(is_prepayment_shipment) as is_prepayment_shipment,
                                        max(is_home_target) as is_home_target,
                                        max(is_pickup_point_target) as is_pickup_point_target,
                                        max(is_delivered_to_home) as is_delivered_to_home,
                                        max(is_delivered_to_pickup_point) as is_delivered_to_pickup_point
                                 from dwh_confidential.f_cot_carrier_shipments
                                 where sk_min_order_created_date between 20220101 and 20221231
                                 group by shipment_id ) as fccs
                                 on ffsoi.shipment_id = fccs.shipment_id

                      -- For delivery service [sk_delivery_carrier]
                      left join dwh_insider.d_delivery_products d_prod
                                on coalesce(ffsoi.sk_delivery_product, fccs.sk_delivery_product) = d_prod.sk_delivery_product

                      -- for carrier name [carrier_name]
                      left join dwh_insider.d_delivery_carriers carr
                                on d_prod.sk_delivery_carrier = carr.sk_delivery_carrier

                      -- For selected fulfillment service [fulfillment_service]
                      left join dwh_insider.d_shp_fulfillment_services f_serv
                                on ffsoi.sk_selected_fulfill_service = f_serv.sk_fulfillment_service

                      -- For delivery service [service_name]
                      left join dwh_insider.d_delivery_services d_serv
                                on ffsoi.sk_delivery_type = d_serv.sk_delivery_service

                      -- For delivery service [service_name]
                      left join dwh_insider.d_delivery_services d_prod_serv
                                on d_prod.sk_delivery_service = d_prod_serv.sk_delivery_service

                      -- For filtering for shop [sk_cfa]
                      left join dwh_insider.d_sales_channels chan
                                on ffsoi.sk_sales_channel = chan.sk_sales_channels

                      -- For getting destination country [country_code]
                      left join (select sk_destination, max(sk_shipping_country) as sk_country
                                 from dwh_confidential.d_destinations
                                 group by 1) as dest
                                on ffsoi.sk_destination = dest.sk_destination

                      -- For filtering Zalando fulfillment [fulfillment_type, warehouse_name]
                      left join dwh_insider.d_stock_location_bp as stock
                                on ffsoi.sk_stock_location_id = stock.sk_stock_location

                      -- For getting destination country [destination_country]
                      left join (select sk_country, country_name_short_en as destination_country
                                 from dwh_confidential.d_country
                                 group by 1, 2) as country
                                on dest.sk_country = country.sk_country

                      left join dwh_insider.d_logistic_provider logistic
                                on ffsoi.sk_logistic_provider = logistic.sk_logistic_provider

            where cast(date_format(ts_est_delivery_to, '%Y%m%d') as integer) between 20220101 and 20221231

             and ffsoi.sk_order_type in (5)
             
             and ffsoi.sk_stock_location_id in (130, 116, 331, 36, 377, 205, 135, 5319099, 167178576, 337994408, 167178575, 89888100, 194682468, 194685072, 194683969)
             and ffsoi.full_cancellation_flag = 0
           
             and chan.sk_cfa in (31, 26, 28)
             and fccs.is_prepayment_shipment in (0)
           )
           Select * from ffsoi_base 
"""
df = pd.read_sql_query(query, connection)
df.head(10)

/tmp/ipykernel_874/1061521932.py:148: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


In [ ]:
df